In [1]:

import os
import json

from langchain import SerpAPIWrapper
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain.utilities import ArxivAPIWrapper
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain import PromptTemplate
from langchain.chains import LLMChain
from typing import Dict, Tuple
from langchain.tools import BaseTool
from pydantic import BaseModel

In [4]:
import boto3
from langchain.llms.bedrock import Bedrock
region="us-east-1"
boto3_bedrock = boto3.client(
    service_name="bedrock-runtime",
    region_name=region
)
llm_model_name = "anthropic.claude-v2"
parameters = {
    "max_tokens_to_sample": 8096,
    "stop_sequences": ["\nObservation"],
    "temperature":0.01,
    "top_p":0.85
}

model_id ="anthropic.claude-instant-v1" if llm_model_name == 'claude-instant' else "anthropic.claude-v2"
llm = Bedrock(model_id=model_id, client=boto3_bedrock, model_kwargs=parameters)
context = """""" 

In [67]:

class AwsOrgServiceInput(BaseModel):
     service: str
     role_type: str

class AwsServiceDiffInput(BaseModel):
     service: str
     region1: str
     region2: str

class aws_org_service(BaseTool):

    name = "aws_org_service"
    description = "aws_org_service(service: str, role_type:str), 用于查询 aws service 服务对应的不同角色负责人，或者查询 aws 员工对应的 service. This tool has two input parameters."
    # args_schema = AwsOrgServiceInput

    def _run(self, input: str) -> str:
        if not input:
            return "tool input parameters error!"
        
        input = json.loads(input)
        service = input["service"]
        role_type = input["role_type"]
        prompt_tmp = """
        你是云服务AWS的智能客服机器人AWSBot

        给你 SSO (Service Specialist Organization) 的组织信息
        {context}

        Job role (角色, 岗位类型) description:
        - GTMS: Go To Market Specialist
        - SS: Specialist Sales
        - SSA: Specialist Solution Architechure
        - TPM: 
        - PM: Project Manager

        Scope means job scope
        service_name equal to business unit

        If the context does not contain the knowleage for the question, truthfully says you does not know.
        Don't put two people's names together. For example, zheng zhang not equal to zheng hao and xueqing not equal to Xueqing Lai

        Find out the most relevant context, and give the answer according to the context
        Skip the preamble; go straight to the point.
        Only give the final answer.
        Do not repeat similar answer.
        使用中文回复，人名不需要按照中文习惯回复

        {service} {role_type} 
        """
        def create_prompt_templete(prompt_template):
            PROMPT = PromptTemplate(
                template=prompt_template,
                input_variables=["context",'service','role_type']
            )
            return PROMPT
        prompt = create_prompt_templete(prompt_tmp) 
        llmchain = LLMChain(llm=llm,verbose=False,prompt = prompt)
        answer = llmchain.run({'service':service, "role_type": role_type, "context": context}).strip()

        return answer

    
class aws_service_difference(BaseTool):

    name = "aws service diff"
    description = "用于查询aws 服务的差异"
    args_schema = AwsServiceDiffInput
    
    def _run(self, input: str) -> str:
        return "aws service 差异查询"

In [63]:
tools = [aws_org_service(), aws_service_difference()]

In [64]:
from langchain.tools.render import render_text_description
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain import hub

# prompt = hub.pull("hwchase17/react")
react_prompt_template = ("Answer the following questions as best you can. "
                "You have access to the following tools:\n\n"
                "{tools}\n\n"
                "Use the following format:\n\n"
                "Question: the input question you must answer\n"
                "Thought: you should always think about what to do\n"
                "Action: the action to take, should be one of [{tool_names}]\n"
                "Action Input: the input to the action. use json fomate.\n"
                "Observation: the result of the action\n"
                "... (this Thought/Action/Action Input/Observation can repeat N times, but do not repeat ask the same question)\n"
                "Thought: I now know the final answer\n"
                "Final Answer: the final answer to the original input question\n\n"
                "Begin!\n\n"
                "Question: {input}\n"
                "Thought:{agent_scratchpad}"
                )
print(react_prompt_template)
react_prompt = PromptTemplate(
                template=react_prompt_template,
                input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] 
            )
prompt = react_prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

print("#"*50)
print(prompt)
llm_with_stop = llm.bind(stop=["\nObservation"])
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action. use json fomate.
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times, but do not repeat ask the same question)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
##################################################
input_variables=['agent_scratchpad', 'input'] partial_variables={'tools': 'aws_org_service: aws_org_service(service: str, role_type:str), 用于查询 aws service 服务对应的不同角色负责人，或者查询 aws 员工对应的 service. This tool has two input parameters.\naws service 差异查询: 用于查询aws 服务的差异', 'tool_names': 'aws_org_service, aws service 差异查询

In [65]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [66]:
agent_executor.invoke(
    {
        "input": "sagemaker的相关问题联系谁？"
    }
)



> Entering new AgentExecutor chain...
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
[]
****************** bedrock prompt *****************
Answer the following questions as best you can. You have access to the following tools:

aws_org_service: aws_org_service(service: str, role_type:str), 用于查询 aws service 服务对应的不同角色负责人，或者查询 aws 员工对应的 service. This tool has two input parameters.
aws service 差异查询: 用于查询aws 服务的差异

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [aws_org_service, aws service 差异查询]
Action Input: the input to the action. use json fomate.
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times, but do not repeat ask the same question)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: sagemaker的相关问题联系谁？
Thought:
****************** end be

{'input': 'sagemaker的相关问题联系谁？', 'output': '根据提供的组织信息,SageMaker的GTMS是Yinuo He。'}